In [1]:
# pip install pymysql

In [2]:
# pip install faker

In [3]:
from faker import Faker
import pymysql
import random
from datetime import datetime, timedelta

# Database connection details
DB_HOST = "127.0.0.1"
DB_USER = "root"
DB_PASSWORD = "Shadow1794"
DB_NAME = "rasakopi"
DB_PORT = 3307

# Initialize Faker
fake = Faker()

def connect_to_db():
    """Establish a connection to the database."""
    try:
        connection = pymysql.connect(
            host=DB_HOST,
            user=DB_USER,
            password=DB_PASSWORD,
            database=DB_NAME,
            port=DB_PORT,
        )
        return connection
    except pymysql.MySQLError as e:
        print(f"Error connecting to MySQL: {e}")
        return None

def create_tables(connection):
    """Create tables based on the ERD."""
    queries = [
        """
        CREATE TABLE IF NOT EXISTS Menu (
            id VARCHAR(50) PRIMARY KEY,
            name VARCHAR(100) NOT NULL,
            price FLOAT NOT NULL,
            description TEXT,
            category VARCHAR(50),
            image_uri TEXT,
            qty INT NOT NULL,
            is_active BOOLEAN NOT NULL DEFAULT TRUE
        );
        """,
        """
        CREATE TABLE IF NOT EXISTS `Table` (
            id VARCHAR(50) PRIMARY KEY,
            min_capacity INT NOT NULL,
            max_capacity INT NOT NULL,
            image_uri TEXT,
            description TEXT,
            no_table VARCHAR(50) UNIQUE NOT NULL,
            is_outdoor BOOLEAN NOT NULL DEFAULT FALSE,
            is_active BOOLEAN NOT NULL DEFAULT TRUE,
            barcode VARCHAR(100)
        );
        """,
        """
        CREATE TABLE IF NOT EXISTS Supplier (
            id VARCHAR(50) PRIMARY KEY,
            name VARCHAR(100) NOT NULL,
            contact VARCHAR(50) NOT NULL,
            type ENUM('Syrup', 'Beans', 'Etc') NOT NULL,
            price FLOAT NOT NULL,
            unit VARCHAR(50) NOT NULL,
            shipping_fee FLOAT NOT NULL,
            address TEXT NOT NULL,
            product_name VARCHAR(100),
            is_active BOOLEAN NOT NULL DEFAULT TRUE
        );
        """,
        """
        CREATE TABLE IF NOT EXISTS Reservasi (
            id VARCHAR(50) PRIMARY KEY,
            reserve_by VARCHAR(100) NOT NULL,
            comunity INT,
            phone_number VARCHAR(30) NOT NULL,
            note TEXT,
            start DATETIME NOT NULL,
            end DATETIME NOT NULL
        );
        """,
        """
        CREATE TABLE IF NOT EXISTS Detail_Reservasi (
            id VARCHAR(50) PRIMARY KEY,
            reservasi_id VARCHAR(50) NOT NULL,
            meja_id VARCHAR(50) NOT NULL,
            FOREIGN KEY (reservasi_id) REFERENCES Reservasi(id) ON DELETE CASCADE,
            FOREIGN KEY (meja_id) REFERENCES `Table`(id) ON DELETE CASCADE
        );
        """,
        """
        CREATE TABLE IF NOT EXISTS `Order` (
            id VARCHAR(50) PRIMARY KEY,
            reservasi_id VARCHAR(50),
            order_by VARCHAR(100) NOT NULL,
            phone_number VARCHAR(30) NOT NULL,
            FOREIGN KEY (reservasi_id) REFERENCES Reservasi(id) ON DELETE SET NULL
        );
        """,
        """
        CREATE TABLE IF NOT EXISTS Order_Detail (
            id VARCHAR(50) PRIMARY KEY,
            order_id VARCHAR(50) NOT NULL,
            menu_id VARCHAR(50) NOT NULL,
            qty INT NOT NULL,
            price FLOAT NOT NULL,
            note TEXT,
            FOREIGN KEY (order_id) REFERENCES `Order`(id) ON DELETE CASCADE,
            FOREIGN KEY (menu_id) REFERENCES Menu(id) ON DELETE CASCADE
        );
        """
    ]

    with connection.cursor() as cursor:
        for query in queries:
            cursor.execute(query)
        connection.commit()





In [4]:
from faker import Faker
from datetime import datetime, timedelta
import random
import pymysql

fake = Faker()
uuid = Faker()

def insert_fake_data(connection):
    """Insert fake data into tables with PK and FK synchronization."""
    try:
        with connection.cursor() as cursor:
            # Menyimpan PK untuk referensi FK
            menu_ids = []         # PK untuk tabel Menu
            table_ids = []        # PK untuk tabel Table
            reservasi_ids = []    # PK untuk tabel Reservasi
            order_ids = []        # PK untuk tabel Order
            supplier_ids = []     # PK untuk tabel Supplier

            # Daftar nama menu tetap dan unik
            menu_items = [
                {'name': 'Teh', 'category': 'minuman'},
                {'name': 'Kopi', 'category': 'minuman'},
                {'name': 'Air', 'category': 'minuman'},
                {'name': 'Macha', 'category': 'minuman'},
                {'name': 'Roti', 'category': 'makanan'},
                {'name': 'Makan berat', 'category': 'makanan'},
                {'name': 'Cemilan', 'category': 'makanan'},
            ]
            
            # Insert data berdasarkan daftar menu tetap
            for item in menu_items:
                menu_id = str(uuid.uuid4())  # Generate UUID
                query = """
                INSERT INTO Menu (id, name, price, description, category, image_uri, qty, is_active)
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
                """
                cursor.execute(query, (
                    menu_id,
                    item['name'],  # Nama dari daftar tetap
                    random.randint(5000, 40000),  # Harga acak
                    fake.sentence(),  # Deskripsi acak
                    item['category'],  # Kategori dari daftar tetap
                    fake.image_url(),  # URL gambar acak
                    random.randint(1, 50),  # Kuantitas acak
                    fake.boolean()  # Status aktif acak
                ))
                menu_ids.append(menu_id)  # Simpan PK

            # Insert fake data into Table
            for i in range(15):
                table_id = str(uuid.uuid4())
                query = """
                INSERT INTO `Table` (id, min_capacity, max_capacity, image_uri, description, no_table, is_outdoor, is_active, barcode)
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
                """
                min_cap = random.randint(2, 4)
                max_cap = random.randint(5, 10)
                cursor.execute(query, (
                    table_id,
                    min_cap,
                    max_cap,
                    fake.image_url(),
                    fake.sentence(),
                    f"T-{random.randint(1, 100)}",  # No meja unik
                    fake.boolean(),
                    fake.boolean(),
                    fake.uuid4()
                ))
                table_ids.append(table_id)

            # Insert fake data into Reservasi
            start_date = datetime(2024, 1, 1)
            for i in range(30):
                reservasi_id = str(uuid.uuid4())
                # Tentukan hari reservasi (increment harian)
                reservasi_date = start_date + timedelta(days=i)
                # Random jam (misal dari jam 8 pagi hingga 10 malam)
                random_hour = random.randint(8, 22)
                # Set waktu start dan end dengan menit dan detik nol
                start_time = reservasi_date.replace(hour=random_hour, minute=0, second=0)
                end_time = start_time + timedelta(hours=2)  # Durasi reservasi 2 jam
                query = """
                INSERT INTO Reservasi (id, reserve_by, comunity, phone_number, note, start, end)
                VALUES (%s, %s, %s, %s, %s, %s, %s)
                """
                cursor.execute(query, (
                    reservasi_id,
                    fake.name(),
                    random.randint(0, 1),
                    fake.phone_number(),
                    fake.boolean(),
                    start_time,
                    end_time
                ))
                reservasi_ids.append(reservasi_id)

            # Insert fake data into Detail_Reservasi
            for i in range(40):
                if reservasi_ids and table_ids:
                    query = """
                    INSERT INTO Detail_Reservasi (id, reservasi_id, meja_id)
                    VALUES (%s, %s, %s)
                    """
                    cursor.execute(query, (
                        str(uuid.uuid4()),
                        random.choice(reservasi_ids),  # Pilih reservasi_id yang valid
                        random.choice(table_ids)      # Pilih meja_id yang valid
                    ))

            # Insert fake data into Order
            for _ in range(30):
                if reservasi_ids:
                    order_id = str(uuid.uuid4())
                    query = """
                    INSERT INTO `Order` (id, reservasi_id, order_by, phone_number)
                    VALUES (%s, %s, %s, %s)
                    """
                    cursor.execute(query, (
                        order_id,
                        random.choice(reservasi_ids),  # Pilih reservasi_id yang valid
                        fake.name(),
                        fake.phone_number()
                    ))
                    order_ids.append(order_id)

            # Insert fake data into Order_Detail
            for _ in range(50):
                if order_ids and menu_ids:
                    query = """
                    INSERT INTO Order_Detail (id, order_id, menu_id, qty, price, note)
                    VALUES (%s, %s, %s, %s, %s, %s)
                    """
                    cursor.execute(query, (
                        str(uuid.uuid4()),
                        random.choice(order_ids),  # Pilih order_id yang valid
                        random.choice(menu_ids),  # Pilih menu_id yang valid
                        random.randint(1, 5),
                        round(random.uniform(5.0, 50.0), 2),
                        fake.sentence()
                    ))

            # Insert fake data into Supplier
            for _ in range(15):  # Sesuaikan jumlah supplier
                supplier_id = str(uuid.uuid4())
                query = """
                INSERT INTO Supplier (id, name, contact, type, price, unit, shipping_fee, address, product_name, is_active)
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                """
                cursor.execute(query, (
                    supplier_id,
                    fake.company(),
                    fake.phone_number(),
                    random.choice(['Syrup', 'Beans', 'Etc']),
                    round(random.uniform(10.0, 500.0), 2),  # Harga random
                    random.choice(['kg', 'liters', 'packs']),
                    round(random.uniform(5.0, 50.0), 2),  # Ongkos kirim random
                    fake.address(),
                    fake.word(),
                    fake.boolean()
                ))
                supplier_ids.append(supplier_id)

            # Commit the transaction
            connection.commit()

    except pymysql.MySQLError as e:
        print(f"Error inserting data: {e}")


In [5]:
def main():
    connection = connect_to_db()
    if connection:
        create_tables(connection)
        insert_fake_data(connection)  # Adjust number of records as needed
        print("Fake data inserted successfully.")
        connection.close()

if __name__ == "__main__":
    main()


Fake data inserted successfully.
